## Introduction
In this tutorial, we will be analyzing instances of failure for our FEA code. We will take the example that we used in the analytical performance tutorial with the stoplights and the heavy birds, and see how improper initialization of the FEA solver will not provide a proper solution.

We begin as we did previously where we define functions to initialize our problem. We keep the same fixed left side nodes and apply a load uniformly to the top of the stoplight beam.

In [1]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import numpy as np

def define_sample_problem_geom(ele_type, nx, ny, L, H):
    coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
    return coords, connect

def define_sample_problem_info(ele_type, coords, connect, L, H):
    # Identify boundaries
    boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)

    # Fixed nodes on the left edge
    fixed_nodes = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

    # Assign distributed load on the right boundary
    q = -50
    dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0, q)

    # Assign material properties
    E = 500000.0
    nu = 0.3
    mu = E / (2.0 * (1.0 + nu))
    kappa = E / (3.0 * (1.0 - 2.0 * nu))
    
    material_props = np.array([mu, kappa])
    # Assign artificial displacement field
    displacement = np.zeros((coords.shape))
    for kk in range(0, coords.shape[0]):
        displacement[kk, 0] = coords[kk, 0] * 0.01
    return displacement, material_props, fixed_nodes, dload_info

## Erroneous Initializations

The section where we define element properties and solver steps contains numerous parts where incorrect initialization will cause the code to break. Let us analyze the changes we made to the code and how they would lead to issues.

### Element Initialization

Our first change here is the number of subdivisions that we have for the element. We set this value comically low - to 1 - which divides the element into 2 equal triangular halves. As a result of this, we do not receive any proper information regarding how the beam deforms and which sections experience how much displacement. On the flip side, if we set these values to be too high, we run into the issue of computationally taxing analysis, where it would simply take too long to compute.


### Solver 

We see our second point of failure when trying to pass our properties to the hyperelastic solver. By setting the tolerance to be too high and the number of iterations to 500, it is impossible to know if the FEA code will converge properly or how long it would take to do so. We must be cautious when deciding how carefully we want to analyze our problems and have reasonable expecations on the accuracy of our computed solution. 

It is important to note that matrix assembly must be done correctly as well. If we were to assign incorrect boundary conditions for our structure, we could run into the issue of the stiffness matrix being singular and not solvable. 

In [3]:
ele_type = "D2_nn3_tri"
nx = 1
ny = 1
L = 2000
H = 50.0

# Generate mesh and plot it
coords, connect = define_sample_problem_geom(ele_type, nx, ny, L, H)
displacement, material_props, fixed_nodes, dload_info = define_sample_problem_info(ele_type, coords, connect, L, H)

fname = "Mesh_failure.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)


# run the example to look at the results
K, R = solver_demo.prep_for_matrix_solve(ele_type, coords.T, connect.T, material_props, displacement.T, fixed_nodes, dload_info)

nr_num_steps = 10
nr_print = True

displacements_all, nr_info_all = hyperelastic_solver(
    material_props, 
    ele_type, 
    coords.T, 
    connect.T, 
    fixed_nodes, 
    dload_info, 
    nr_print, 
    nr_num_steps, 
    nr_tol=1e-30, 
    nr_maxit=500)

fname = "failure.gif"
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)

final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

# We need to pick a node near the end of the beam and half of the height
tip_node = 0
for i, (x, y) in enumerate(coords):
    if abs(x - L) < 1e-6 and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break

tip_disp_y = final_disp[tip_node * 2 + 1]

print(f"Computed tip deflection (y): {tip_disp_y:.6f}")


Step 0, load factor = 0.100
Iteration 1, Correction=1.000000e+00, Residual=6.250000e+02, tolerance=1.000000e-30
Iteration 2, Correction=1.820404e-04, Residual=9.175097e+01, tolerance=1.000000e-30
Iteration 3, Correction=3.068177e-10, Residual=3.279183e-06, tolerance=1.000000e-30
Iteration 4, Correction=4.407381e-14, Residual=2.036373e-08, tolerance=1.000000e-30
Iteration 5, Correction=1.655605e-14, Residual=3.554175e-09, tolerance=1.000000e-30
Iteration 6, Correction=5.376619e-14, Residual=3.558398e-09, tolerance=1.000000e-30
Iteration 7, Correction=1.714623e-14, Residual=3.560260e-09, tolerance=1.000000e-30
Iteration 8, Correction=1.649874e-14, Residual=3.550268e-09, tolerance=1.000000e-30
Iteration 9, Correction=4.372769e-14, Residual=2.035629e-08, tolerance=1.000000e-30
Iteration 10, Correction=1.095616e-13, Residual=1.236515e-08, tolerance=1.000000e-30
Iteration 11, Correction=1.650649e-14, Residual=3.550188e-09, tolerance=1.000000e-30
Iteration 12, Correction=1.648994e-14, Residua

### Conclusion

While the FEA code is extremely comprehensive and detailed, without looking into the functions and learning how they are put together, users are prone to running into bugs, errors, or completely incorrect solutions in the first place. Proper breakdown of each file can be seen in part 3 where each python file, tutorial, and more is broken down.